In [5]:
# 1. 데이터 확보
import zipfile
filepath = 'data/glaucoma.zip'
with zipfile.ZipFile(filepath,'r')as f:
    f.extractall('data/')

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
train_data = train_generator.flow_from_directory(
    'data/glaucoma/train/',
    target_size=(224,224),
    batch_size=4,
    class_mode='sparse', # 종속변수에 원핫인코딩 안함
    )
test_generator = ImageDataGenerator()
test_data = test_generator.flow_from_directory(
    'data/glaucoma/test/',
    target_size=(224,224),
    batch_size=4,
    class_mode='sparse'
)

Found 828 images belonging to 3 classes.
Found 320 images belonging to 3 classes.


In [8]:
# 2. 모델구성
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense 
from tensorflow.keras.preprocessing import image # 이미지를 읽어오기

resnet_model = ResNet50(input_shape=(224,224,3),
                        include_top=False) # 컨볼루션과 폴링 레이어만 받아오기\
resnet_model.trainable = True
model = Sequential([
    resnet_model,
    Flatten(),
    Dense(units=1024,activation ='relu'),
    Dense(3,activation='softmax')    
])
model.summary()
# 파라미터 사용 메모리량
# 파라미터수: 126,352,259*4byte / 1024의 제곱 = 482mb
# 배치사이즈 =3 : 3*(224*224*3) * 4byte/ 1024의 제곱 = 19 mb
# 각 layer 층 마다 활성화 값 계산


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[100352,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [15]:
# 3. 학습설정
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy']) #리콜 프리시즌 에큐러시차이 구분
# 4. 학습시키기
model.fit(train_data,validation_data=test_data,
         epochs=20,batch_size=3)



Epoch 1/20


ResourceExhaustedError: in user code:

    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 997, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 579, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 695, in

In [ ]:
img = image.load_img('data/glaucoma/')